# Assignment 1
## Recreate Fama-French (1993) SMB and HML factors 

### Replication Procedure
1) Main data required is monthly returns, market equity, and book equity
    
    a) Use all non-financial firms (shrcd = 11, 12) which intersect the NYSE, AMEX and NASDAQ (exchcd = 1,2,3) from CRSP and the merged COMPUSTAT database for the financial statement data
    
    b) Data starts in 1962, however end of June of 1963 is first year, with t-1 data being from 1962  
    
2) Clean data appropriately
    a) BE is defined as COMPUSTATs book value of stockholders equity + balance-sheet deferred taxes - book value of preferred stock
    
    b) Negative BE firms are excluded

3) Organize necessary variables for each stock:

    a) Compute B/M_t as BE divided ME (both) in December of year t-1   
    
    b) ME_t as of June of year t
    
    c) All above variables are in logs
    
    d) Returns are computed monthly

4) Sort stocks based on Size and B/M
    
    a) B/M divded into Low, Medium and High, with the distinction for low as the lowest 30%, and high the highest 30%
    
    b) Size divided into Small and Big, with the distinction being the NYSE median breakpoint in June of year t
    
5) Compute value-weighted portfolio return from July of year t to June of year t+1, where the portfolio is rebalanced in June of t+1

6) Compute HML factor as the difference between the average portfolio returns for the High, Small and Big portfolios and the Low, Small and Big portfolios

7) Compute the SMB factor as the different between the average portfolio returns for the Small, L, M and H portfolios and the Big, L, M and H portfolios

### Data

**CRSP Data Fields**

PERMNO - unique security identifier

PRC - closing price

SHROUT - shares outstanding

Market Cap = PRC * SHROUT

**COMPUSTAT Data Fields**
North America File:

GVKEY - unique security identifier

IID - security ID

Unique Security Identifier = concat(GVKEY, IID)

EXCHG - Exchange Code


In [63]:
# Imports
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *

#### Link to WRDS

In [64]:
import wrds
conn = wrds.Connection(wrds_username="santoromano")
# conn.create_pgpass_file()

Loading library list...
Done


#### Imports and Cleans COMPUSTAT Data

In [65]:
comp = conn.raw_sql("""
                    select gvkey, datadate, at, pstkl, txditc, pstkrv, seq, pstk
                    from comp.funda
                    where indfmt='INDL' 
                    and datafmt='STD'
                    and popsrc='D'
                    and consol='C'
                    and datadate >= '01/01/1959'
                    """, 
                    date_cols=['datadate'])
comp['year'] = comp['datadate'].dt.year

In [66]:
# create preferrerd stock
comp['ps']=np.where(comp['pstkrv'].isnull(), comp['pstkl'], comp['pstkrv'])
comp['ps']=np.where(comp['ps'].isnull(),comp['pstk'], comp['ps'])
comp['ps']=np.where(comp['ps'].isnull(),0,comp['ps'])
comp['txditc']=comp['txditc'].fillna(0)

In [67]:
# create book equity
comp['be']=comp['seq']+comp['txditc']-comp['ps']
comp['be']=np.where(comp['be']>0, comp['be'], np.nan)

In [68]:
# number of years in Compustat
comp=comp.sort_values(by=['gvkey','datadate'])
comp['count']=comp.groupby(['gvkey']).cumcount()

comp=comp[['gvkey','datadate','year','be','count']]

In [70]:
# Stores cleaned COMPUSTAT data locally
comp.to_pickle("compustat_data.pkl")

#### Imports and Cleans CRSP Data

In [69]:
crsp_m = conn.raw_sql("""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1959' and '12/31/2022'
                      and b.exchcd between 1 and 3
                      """, date_cols=['date']) 

# change variable format to int
crsp_m[['permco','permno','shrcd','exchcd']]=crsp_m[['permco','permno','shrcd','exchcd']].astype(int)

# Line up date to be end of month
crsp_m['jdate']=crsp_m['date']+MonthEnd(0)

In [71]:
# add delisting return
dlret = conn.raw_sql("""
                     select permno, dlret, dlstdt 
                     from crsp.msedelist
                     """, date_cols=['dlstdt'])

dlret.permno=dlret.permno.astype(int)
#dlret['dlstdt']=pd.to_datetime(dlret['dlstdt'])
dlret['jdate']=dlret['dlstdt']+MonthEnd(0)

crsp = pd.merge(crsp_m, dlret, how='left',on=['permno','jdate'])
crsp['dlret']=crsp['dlret'].fillna(0)
crsp['ret']=crsp['ret'].fillna(0)

# retadj factors in the delisting returns
crsp['retadj']=(1+crsp['ret'])*(1+crsp['dlret'])-1

# calculate market equity
crsp['me']=crsp['prc'].abs()*crsp['shrout'] 
crsp=crsp.drop(['dlret','dlstdt','prc','shrout'], axis=1)
crsp=crsp.sort_values(by=['jdate','permco','me'])

In [72]:
# Aggreate Market Cap

# sum ME across different permno for same permco for a given date 
crsp_summe = crsp.groupby(['jdate','permco'])['me'].sum().reset_index()

# largest mktcap within a permco/date
crsp_maxme = crsp.groupby(['jdate','permco'])['me'].max().reset_index()

# join by jdate/maxme to find the permno
crsp1=pd.merge(crsp, crsp_maxme, how='inner', on=['jdate','permco','me'])

# drop me column and replace with the sum me
crsp1=crsp1.drop(['me'], axis=1)

# join with sum of me to get the correct market cap info
crsp2=pd.merge(crsp1, crsp_summe, how='inner', on=['jdate','permco'])

# sort by permno and date and also drop duplicates
crsp2=crsp2.sort_values(by=['permno','jdate']).drop_duplicates()

In [73]:
# keep December market cap
crsp2['year']=crsp2['jdate'].dt.year
crsp2['month']=crsp2['jdate'].dt.month
decme=crsp2[crsp2['month']==12]
decme=decme[['permno','date','jdate','me','year']].rename(columns={'me':'dec_me'})

# July to June dates
crsp2['ffdate']=crsp2['jdate']+MonthEnd(-6)
crsp2['ffyear']=crsp2['ffdate'].dt.year
crsp2['ffmonth']=crsp2['ffdate'].dt.month
crsp2['1+retx']=1+crsp2['retx']
crsp2=crsp2.sort_values(by=['permno','date'])

In [74]:
# cumret by stock
crsp2['cumretx']=crsp2.groupby(['permno','ffyear'])['1+retx'].cumprod()

# lag cumret
crsp2['lcumretx']=crsp2.groupby(['permno'])['cumretx'].shift(1)

# lag market cap
crsp2['lme']=crsp2.groupby(['permno'])['me'].shift(1)

# if first permno then use me/(1+retx) to replace the missing value
crsp2['count']=crsp2.groupby(['permno']).cumcount()
crsp2['lme']=np.where(crsp2['count']==0, crsp2['me']/crsp2['1+retx'], crsp2['lme'])

# baseline me
mebase=crsp2[crsp2['ffmonth']==1][['permno','ffyear', 'lme']].rename(columns={'lme':'mebase'})

# merge result back together
crsp3=pd.merge(crsp2, mebase, how='left', on=['permno','ffyear'])
crsp3['wt']=np.where(crsp3['ffmonth']==1, crsp3['lme'], crsp3['mebase']*crsp3['lcumretx'])

decme['year']=decme['year']+1
decme=decme[['permno','year','dec_me']]

In [75]:
# Info as of June
crsp3_jun = crsp3[crsp3['month']==6]

crsp_jun = pd.merge(crsp3_jun, decme, how='inner', on=['permno','year'])
crsp_jun=crsp_jun[['permno','date', 'jdate', 'shrcd','exchcd','retadj','me','wt','cumretx','mebase','lme','dec_me']]
crsp_jun=crsp_jun.sort_values(by=['permno','jdate']).drop_duplicates()

In [76]:
# Stores cleaned CRSP data locally
crsp.to_pickle("crsp_data.pkl")

#### Merge CRSP and Compustat
Merge with CCM Link Table

##### Reads COMPUSTAT and CRSP Data locally

In [ ]:
ccm=conn.raw_sql("""
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

ccm1=pd.merge(comp[['gvkey','datadate','be', 'count']],ccm,how='left',on=['gvkey'])
ccm1['yearend']=ccm1['datadate']+YearEnd(0)
ccm1['jdate']=ccm1['yearend']+MonthEnd(6)

#### Ken French Data